In [0]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import os

In [0]:
from sklearn.metrics import make_scorer
from sklearn.svm import SVC

In [4]:
!wget https://github.com/thedenaas/hse_seminars/blob/master/2019/exam/exam_data.zip?raw=true

--2020-03-24 10:29:58--  https://github.com/thedenaas/hse_seminars/blob/master/2019/exam/exam_data.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/thedenaas/hse_seminars/raw/master/2019/exam/exam_data.zip [following]
--2020-03-24 10:29:58--  https://github.com/thedenaas/hse_seminars/raw/master/2019/exam/exam_data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thedenaas/hse_seminars/master/2019/exam/exam_data.zip [following]
--2020-03-24 10:29:58--  https://raw.githubusercontent.com/thedenaas/hse_seminars/master/2019/exam/exam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.

In [6]:
os.listdir()

['.config', 'exam_data.zip?raw=true', 'sample_data']

In [7]:
!unzip exam_data.zip\?raw\=true

Archive:  exam_data.zip?raw=true
  inflating: test.csv                
  inflating: train.csv               


In [0]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [0]:
df_train['target'] = (df_train['target'] == 5).astype(np.int)
df_test['target'] = (df_test['target'] == 5).astype(np.int)

In [10]:
df_train.shape

(48192, 3)

In [11]:
df_train.head()

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,1
1,Excellent service - very approachable and prof...,Excellent Service,0
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",1
3,"a little noisy, there was a false fire alarm a...","nice hotel,",0
4,Place had too many animals and I'm allergic to...,Experience,0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
from sklearn.metrics import f1_score

In [0]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
vec = TfidfVectorizer().fit(df_train['review'])
vec1 = TfidfVectorizer().fit(df_train['title'])

In [0]:
X_train = sp.hstack([vec.transform(df_train['review']),
                    vec1.transform(df_train['title'])])
X_test = sp.hstack([vec.transform(df_test['review']),
                   vec1.transform(df_test['title'])])
y_train = df_train['target']
y_test = df_test['target']

In [0]:
clf = SGDClassifier(random_state=42, n_jobs=-1)
param_grid = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [10**(-i) for i in range(1,7)]
}

In [0]:
grid_search = GridSearchCV(clf, param_grid, n_jobs=-1,
                          scoring=make_scorer(f1_score))

In [36]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=-1,
                                     penalty='l2', power_t=0.5, random_state=42,
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06],
                         'loss': ['hinge', 'log', 'modified_huber',
                                  'squared_hinge', 'perceptron'],
  

In [37]:
f1_score(y_test, grid_search.predict(X_test))

0.7411988582302569

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
clf1 = RandomForestClassifier(random_state=42)

In [27]:
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [28]:
f1_score(y_test, clf1.predict(X_test))

0.6863694267515923

In [0]:
clf2 = SVC(random_state=42)
param_grid2 = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

grid_search2 = GridSearchCV(clf2, param_grid2, n_jobs=-1,
                           scoring=make_scorer(f1_score))

In [0]:
grid_search2.fit(X_train, y_train)